In [ ]:
'''
il simbolo ! in google colab permette di eseguire comandi di shell direttamente da un notebook.
Esegui il comando shell unrar per estrarre (x) tutti i file e le directory,
mantenendo la loro struttura originale, dall'archivio dataset.rar e salvali nella cartella dataset
(creandola se non esiste) nella directory di lavoro corrente.
'''
!unrar x dataset.rar dataset


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from dataset.rar

Creating    dataset                                                   OK
Creating    dataset/groundtruth01                                     OK
Extracting  dataset/groundtruth01/unogt100.jpg                             0%  OK 
Extracting  dataset/groundtruth01/unogt140.jpg                             0%  OK 
Extracting  dataset/groundtruth01/unogt141.jpg                             0%  OK 
Extracting  dataset/groundtruth01/unogt142.jpg                             0%  OK 
Extracting  dataset/groundtruth01/unogt143.jpg                             0%  OK 
Extracting  dataset/groundtruth01/unogt144.jpg                             0%  OK 
Extracting  dataset/groundtruth01/unogt145.jpg                             0%  OK 
Extracting  dataset/groundtruth01/unogt146.jpg                             0%  OK 
Extracting  d

In [13]:
'''
Utilizzando il gestore di pacchetti Python pip, scarica e installa nell'ambiente Python
corrente le seguenti librerie: opencv-python-headless (per la computer vision senza GUI),
numpy (per il calcolo numerico), scikit-learn (per il machine learning), pyyaml
(per la gestione di file YAML) e ultralytics (per i modelli YOLO e altre utility di deep learning).
'''
!pip install opencv-python-headless numpy scikit-learn pyyaml ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 129.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 883.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [ ]:
# il comando import importa moduli e pacchetti in Python.
'''
questo script serve a preprocessare i dati per l'addestramento di un modello YOLO (You Only Look Once)
per la rilevazione di oggetti, in questo caso per la tiroide.
'''
#Importa la libreria OpenCV (cv2), utilizzata per l'elaborazione di immagini e video.
import cv2
#Importa la libreria NumPy e le assegna l'alias 'np'. NumPy è fondamentale per il calcolo numerico e la manipolazione di array.
import numpy as np
#Importa il modulo 'os', che fornisce funzioni per interagire con il sistema operativo, come la gestione di percorsi e directory.
import os
#Importa il modulo 'glob', utilizzato per trovare tutti i nomi di percorso che corrispondono a un pattern specificato.
import glob
#Dalla libreria Scikit-learn (sklearn), importa la funzione 'train_test_split', utile per suddividere i dati in set di addestramento e validazione.
from sklearn.model_selection import train_test_split
#Importa la libreria 'yaml', utilizzata per leggere e scrivere file in formato YAML (YAML Ain't Markup Language), spesso usato per file di configurazione.
import yaml
#Importa il modulo 'shutil', che offre operazioni su file e collezioni di file di alto livello, come la copia e la rimozione di alberi di directory. Il commento originale indica già il suo uso per copiare i file.
import shutil
#Importa il modulo 're', che fornisce supporto per le espressioni regolari, utili per la ricerca e la manipolazione di stringhe basate su pattern.
import re

# --- CONFIGURAZIONE ---
base_dataset_folder = 'dataset' # La cartella che contiene 'originali01', 'groundtruth01' etc.
yolo_dataset_folder = 'yolo_thyroid_dataset' # Cartella di output con file postprocess per training di YOLO

# Definisci la regola per trovare la maschera dall'immagine originale
# Esempio: se l'immagine è 'dataset/originali01/uno100.jpg', la maschera è 'dataset/groundtruth01/unogt100.jpg'
def get_mask_path(image_path):
    '''
   Firma della funzione: get_mask_path(image_path)
   - def: Parola chiave che definisce una funzione.
   - get_mask_path: Nome della funzione.
   - image_path: Parametro in input. È una stringa che rappresenta il percorso completo di un file immagine originale.
   Scopo: Questa funzione costruisce e restituisce il percorso presunto del file maschera corrispondente all'immagine fornita.
          Si basa su una convenzione specifica di denominazione e struttura delle cartelle:
          1. Estrae il numero identificativo dalla cartella dell'immagine originale (es. '01' da 'originali01').
          2. Usa questo numero per formare il nome della cartella delle maschere (es. 'groundtruth01').
          3. Modifica il nome del file immagine (es. sostituendo 'uno' con 'unogt') per ottenere il nome del file maschera.
   Ritorna: Una stringa con il percorso completo al file maschera se la logica ha successo, altrimenti None se non è possibile determinare il percorso (es. se il numero della cartella non viene trovato).
    '''
    #Divide il percorso dell'immagine ('image_path') in una lista di componenti, utilizzando il separatore di directory specifico del sistema operativo (os.sep, es. '/' o '\').
    parts = image_path.split(os.sep)
    #Estrae l'ultimo elemento della lista 'parts', che corrisponde al nome del file con estensione (es. 'uno100.jpg').
    filename = parts[-1]
    # Estrai il numero dalla cartella originale (es. '01' da 'originali01')
    folder_name = parts[-2]
    #Utilizza un'espressione regolare (re.search) per trovare una sequenza di una o più cifre ('\d+') alla fine ('$') del nome della cartella ('folder_name').
    folder_num_match = re.search(r'\d+$', folder_name)
    #Controlla se l'espressione regolare ha trovato una corrispondenza (cioè, se è stato trovato un numero alla fine del nome della cartella).
    if folder_num_match:
        #Se una corrispondenza è stata trovata, 'folder_num_match.group()' restituisce la stringa corrispondente (es. '01').
        folder_num = folder_num_match.group()
        #Costruisce il nome della cartella delle maschere anteponendo 'groundtruth' al numero estratto (es. 'groundtruth01').
        mask_folder = f'groundtruth{folder_num}'
        #Crea il nome del file maschera sostituendo 'uno' con 'unogt' nel nome del file originale.
        mask_filename = filename.replace('uno', 'unogt')
        #Combina il percorso base del dataset, la cartella della maschera calcolata e il nome del file maschera calcolato per formare il percorso completo del file maschera, e lo restituisce.
        return os.path.join(base_dataset_folder, mask_folder, mask_filename)
    else:
        #Se l'espressione regolare non ha trovato un numero alla fine del nome della cartella.
        print(f"Attenzione: Impossibile determinare la cartella groundtruth per {image_path}")
        #Restituisce None per segnalare che il percorso della maschera non è stato trovato.
        return None #

# Classe target (solo una: tiroide)
class_index = 0
class_name = 'thyroid'
# --- FINE CONFIGURAZIONE ---

print(f"Cartella dataset originale: {os.path.abspath(base_dataset_folder)}")
print(f"Cartella dataset YOLO di output: {os.path.abspath(yolo_dataset_folder)}")

# Pulisci la cartella di output se esiste già (opzionale, ma utile in Colab)
if os.path.exists(yolo_dataset_folder):
    print(f"Rimuovo la cartella YOLO esistente: {yolo_dataset_folder}")
    shutil.rmtree(yolo_dataset_folder)
'''
Crea le cartelle necessarie per il training di YOLO
le cartelle sono:
- images/train: per le immagini di addestramento
- images/val: per le immagini di validazione
- labels/train: per le etichette di addestramento
- labels/val: per le etichette di validazione
'''
os.makedirs(os.path.join(yolo_dataset_folder, 'images', 'train'), exist_ok=True)
os.makedirs(os.path.join(yolo_dataset_folder, 'images', 'val'), exist_ok=True)
os.makedirs(os.path.join(yolo_dataset_folder, 'labels', 'train'), exist_ok=True)
os.makedirs(os.path.join(yolo_dataset_folder, 'labels', 'val'), exist_ok=True)

print("Struttura cartelle YOLO creata.")

#Inizializza una lista vuota chiamata 'all_image_paths' che conterrà i percorsi di tutte le immagini originali trovate.
all_image_paths = []
# Trova tutte le immagini JPG nelle cartelle 'originali*'
#Itera su tutte le cartelle all'interno di 'base_dataset_folder' il cui nome inizia con 'originali' (es. 'originali01', 'originali02').
for original_folder in glob.glob(os.path.join(base_dataset_folder, 'originali*')):
    #Per ogni cartella 'original_folder' trovata, cerca tutti i file con estensione '.jpg' e aggiunge i loro percorsi alla lista 'all_image_paths'.
    all_image_paths.extend(glob.glob(os.path.join(original_folder, '*.jpg')))

print(f"Trovate {len(all_image_paths)} immagini originali.")

#Inizializza una lista vuota chiamata 'image_mask_pairs' per memorizzare le coppie di percorsi (immagine originale, maschera corrispondente).
image_mask_pairs = []
#Itera su ciascun percorso di immagine trovato e memorizzato in 'all_image_paths'.
for img_path in all_image_paths:
    #Chiama la funzione 'get_mask_path' per ottenere il percorso della maschera corrispondente all'immagine corrente.
    mask_path = get_mask_path(img_path)
    # Aggiungi solo se get_mask_path ha restituito un percorso valido e se esiste
    #Controlla due condizioni: 1) 'mask_path' non è None (cioè la funzione ha restituito un percorso) E 2) il file specificato da 'mask_path' esiste effettivamente.
    if mask_path and os.path.exists(mask_path):
        #Se entrambe le condizioni sono vere, aggiunge una tupla (coppia) contenente il percorso dell'immagine e il percorso della maschera alla lista 'image_mask_pairs'.
        image_mask_pairs.append((img_path, mask_path))
    elif mask_path:
        #Se 'mask_path' è stato restituito (non è None) ma il file non esiste (la condizione 'os.path.exists(mask_path)' era falsa).
        print(f"Attenzione: Maschera non trovata per {img_path} al percorso calcolato {mask_path}")

print(f"Trovate {len(image_mask_pairs)} coppie immagine-maschera valide.")

#Controlla se la lista 'image_mask_pairs' è vuota (cioè, non sono state trovate coppie valide).
if not image_mask_pairs:
    print("Nessuna coppia immagine-maschera trovata. Controlla i percorsi e la funzione get_mask_path.")
#Se almeno una coppia immagine-maschera è stata trovata.
else:
    #Suddivide la lista 'image_mask_pairs' in due sotto-liste: 'train_pairs' (80% dei dati) e 'val_pairs' (20% dei dati).
    #'test_size=0.20' specifica la proporzione del dataset da allocare al set di validazione.
    #'random_state=42' assicura che la suddivisione sia riproducibile; usando lo stesso stato casuale si otterrà sempre la stessa suddivisione.
    train_pairs, val_pairs = train_test_split(image_mask_pairs, test_size=0.20, random_state=42)
    print(f"Suddivisione: {len(train_pairs)} training, {len(val_pairs)} validation.")

def process_dataset_split(pairs, split_name):
    '''
   Firma della funzione: process_dataset_split(pairs, split_name)
   - process_dataset_split: Nome della funzione.
   - pairs: Parametro in input. È una lista di tuple, dove ogni tupla contiene due stringhe: (percorso_immagine_originale, percorso_maschera_corrispondente).
   - split_name: Parametro in input. È una stringa che indica il nome del set di dati che si sta processando (es. "train" o "val").
   Scopo: Questa funzione elabora un insieme di coppie immagine-maschera per un determinato split (addestramento o validazione).
          Per ogni coppia:
          1. Costruisce nuovi nomi per l'immagine e il file di etichetta, includendo un numero derivato dalla sottocartella originale per garantire unicità.
          2. Copia l'immagine originale nella sottocartella 'images/<split_name>' della directory YOLO con il nuovo nome.
          3. Legge l'immagine della maschera, la binarizza e trova i contorni della regione di interesse (es. la tiroide).
          4. Normalizza le coordinate dei punti del contorno rispetto alle dimensioni dell'immagine (valori tra 0 e 1).
          5. Salva i contorni normalizzati in un file di testo (formato etichetta YOLO) nella sottocartella 'labels/<split_name>' con il nuovo nome.
   Ritorna: Nulla (la funzione modifica il filesystem creando file e cartelle).
   '''
    print(f"\nInizio processamento per lo split: {split_name}")
    #Inizializza un contatore per le immagini processate con successo.
    processed_count = 0
    #Inizializza un contatore per i file saltati a causa dell'impossibilità di estrarre il numero della sottocartella.
    skipped_no_folder_num = 0

    #Itera su ogni tupla (percorso immagine, percorso maschera) nella lista 'pairs'.
    for img_path, mask_path in pairs:
        #Inizia un blocco try-except per gestire potenziali errori durante l'elaborazione di una singola coppia immagine-maschera.
        try:
            # --- Estrai informazioni per il nuovo nome ---
            #Estrae il nome base del file originale dell'immagine (es. "uno100.jpg") dal suo percorso completo.
            original_base_filename = os.path.basename(img_path)
            #Separa il nome base del file nella sua parte nominale (es. "uno100") e la sua estensione (es. ".jpg").
            name_part, extension = os.path.splitext(original_base_filename)

            # Divide il percorso completo dell'immagine originale nei suoi componenti.
            parts = img_path.split(os.sep)
            #Estrae il nome della cartella che contiene l'immagine (es. 'originali01').
            folder_name = parts[-2]
            #Utilizza un'espressione regolare per trovare una sequenza di cifre alla fine del nome della cartella.
            folder_num_match = re.search(r'\d+$', folder_name)

            #Se l'espressione regolare non trova un numero alla fine del nome della cartella.
            if not folder_num_match:
                print(f"Attenzione: Impossibile estrarre il numero dalla cartella '{folder_name}' per l'immagine {img_path}. Salto questo file.")
                #Incrementa il contatore dei file saltati per questo motivo.
                skipped_no_folder_num += 1
                #Interrompe l'iterazione corrente del ciclo e passa alla prossima coppia immagine-maschera.
                continue

            #Estrae la stringa numerica trovata (es. '01').
            subfolder_num_str = folder_num_match.group()

            # Costruisci i nuovi nomi dei file
            new_base_filename = f"{name_part}_{subfolder_num_str}{extension}" # Es: uno140_01.jpg
            #Costruisce il nuovo nome del file etichetta, usando la stessa base del nuovo nome immagine ma con estensione ".txt" (es. "uno140_01.txt").
            new_label_filename = f"{name_part}_{subfolder_num_str}.txt"

            # --- Processa Immagine ---
            # Legge l'immagine originale dal percorso 'img_path' utilizzando OpenCV. 'img' sarà un array NumPy.
            img = cv2.imread(img_path)
            #Controlla se il caricamento dell'immagine è fallito (imread restituisce None in caso di errore).
            if img is None:
                print(f"Errore nel leggere l'immagine: {img_path}")
                #Salta all'iterazione successiva del ciclo.
                continue
            #Estrae altezza ('img_height') e larghezza ('img_width') dell'immagine dalle dimensioni dell'array NumPy ('img.shape').
            img_height, img_width = img.shape[:2]

            # Definisci il percorso di destinazione con il NUOVO nome
            yolo_img_path = os.path.join(yolo_dataset_folder, 'images', split_name, new_base_filename)
            #Copia il file immagine originale ('img_path') nella nuova posizione ('yolo_img_path'). 'copy2' tenta di preservare anche i metadati del file.
            shutil.copy2(img_path, yolo_img_path)

            # --- Processa Maschera e Crea Etichetta .txt ---
            #Legge l'immagine della maschera dal percorso 'mask_path' in scala di grigi (cv2.IMREAD_GRAYSCALE).
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            #Controlla se il caricamento della maschera è fallito.
            if mask is None:
                print(f"Errore nel leggere la maschera: {mask_path} (associata a {img_path})")
                #Se l'immagine era già stata copiata ma la sua maschera non è leggibile, rimuove l'immagine copiata per mantenere la coerenza.
                if os.path.exists(yolo_img_path): os.remove(yolo_img_path)
                #Salta all'iterazione successiva.
                continue

            #Applica una binarizzazione alla maschera. I pixel con valore > 127 diventano 255 (bianco), gli altri 0 (nero). Il primo valore restituito ('_') è la soglia usata, non ci serve quindi usiamo il simbolo '_' per ignorarlo.
            _, mask_binary = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

            #Trova i contorni nell'immagine della maschera binarizzata.
            # - 'mask_binary': Immagine sorgente (binaria).
            # - 'cv2.RETR_EXTERNAL': Modalità di recupero dei contorni; recupera solo i contorni esterni.
            # - 'cv2.CHAIN_APPROX_SIMPLE': Metodo di approssimazione dei contorni; comprime segmenti orizzontali, verticali e diagonali lasciando solo i loro punti finali.
            # 'contours' è una lista di tutti i contorni trovati, ognuno è un array NumPy di coordinate (x,y) dei punti del contorno.
            # 'hierarchy' contiene informazioni sulla topologia dei contorni (non usata attivamente in questo script).
            contours, hierarchy = cv2.findContours(mask_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            #Costruisce il percorso completo per il file di etichetta .txt nella struttura YOLO, usando il nuovo nome file.
            yolo_label_path = os.path.join(yolo_dataset_folder, 'labels', split_name, new_label_filename)

            #Apre il file di etichetta in modalità scrittura ('w'). Il costrutto 'with' assicura che il file venga chiuso automaticamente. 'f' è l'oggetto file.
            with open(yolo_label_path, 'w') as f:
                #Inizializza un contatore per i contorni validi (quelli che soddisfano i criteri di dimensione/area) trovati in questa maschera.
                valid_contours_found = 0
                #Itera su ciascun contorno trovato nell'immagine della maschera.
                for contour in contours:
                    #Applica filtro su punti E area (opzionale ma consigliato)
                    #Definisce un'area minima (in pixel) affinché un contorno sia considerato valido.
                    min_contour_area = 5 # Ignora contorni con area < 5 pixel
                    #Controlla se il contorno è valido:
                    # 1. 'contour.size >= 6': Verifica che il contorno abbia almeno 3 punti (poiché ogni punto ha 2 coordinate (x,y), quindi 3*2=6 valori). Questo è necessario per definire una forma poligonale.
                    # 2. 'cv2.contourArea(contour) > min_contour_area': Verifica che l'area del contorno sia maggiore della soglia minima definita.
                    if contour.size >= 6 and cv2.contourArea(contour) > min_contour_area:
                        #'contour.flatten()' converte l'array del contorno (che potrebbe essere [[x1,y1],[x2,y2],...]) in un array monodimensionale [x1,y1,x2,y2,...].
                        normalized_contour = contour.flatten().astype(float)
                        #Normalizza le coordinate x (elementi a indice pari: 0, 2, 4, ...) dividendo per la larghezza dell'immagine. Il risultato è tra 0 e 1.
                        normalized_contour[0::2] /= img_width
                        #normalizza le coordinate y (elementi a indice dispari: 1, 3, 5, ...) dividendo per l'altezza dell'immagine. Il risultato è tra 0 e 1.
                        normalized_contour[1::2] /= img_height
                        #Assicura che tutte le coordinate normalizzate siano strettamente comprese tra 0 e 1 (es. 0.00001 e 0.99999). Evita valori esattamente uguali a 0.0 o 1.0 che potrebbero causare problemi con alcune implementazioni YOLO.
                        normalized_contour = np.clip(normalized_contour, 0.00001, 0.99999)
                        #Converte ogni coordinata normalizzata in una stringa formattata con 6 cifre decimali (es. "0.123456") e poi unisce tutte queste stringhe con uno spazio come separatore.
                        contour_str = ' '.join(map(lambda x: f"{x:.6f}", normalized_contour))
                        #Scrive una riga nel file di etichetta. La riga contiene: l'indice della classe ('class_index'), seguito da uno spazio, seguito dalle coordinate normalizzate del contorno, e termina con un carattere di nuova riga ('\n').
                        f.write(f"{class_index} {contour_str}\n")
                        #Incrementa il contatore dei contorni validi scritti su file per questa immagine.
                        valid_contours_found += 1
                #Se sono stati trovati dei contorni ('contours' non è vuoto) ma nessuno di essi è risultato valido ('valid_contours_found' è 0).
                if contours and valid_contours_found == 0:
                     print(f"Info: Trovati contorni in {mask_path}, ma nessuno era valido (>=3 punti e area > {min_contour_area}). File {new_label_filename} creato vuoto.")
                #Se non è stato trovato alcun contorno nell'immagine della maschera.
                elif not contours:
                     print(f"Info: Nessun contorno trovato in {mask_path}. File {new_label_filename} creato vuoto.")
                     #Non fa nulla; il file di etichetta rimarrà vuoto, che è il comportamento corretto se non ci sono oggetti.
                     pass

            #Incrementa il contatore delle immagini processate con successo in questo split.
            processed_count += 1
            #Controlla se il numero di immagini processate è un multiplo di 50.
            if processed_count % 50 == 0: # Stampa ogni 50 immagini
                print(f"  Processate {processed_count}/{len(pairs)} immagini per lo split {split_name}...")

        #Se si verifica un qualsiasi altro errore non gestito specificamente nel blocco 'try'. 'e' contiene l'oggetto eccezione.
        except Exception as e:
            print(f"Errore grave durante il processamento di {img_path} o {mask_path}: {e}")

    print(f"Fine processamento per lo split: {split_name}. Immagini processate correttamente: {processed_count}.")
    #Se alcuni file sono stati saltati a causa del problema con il numero della sottocartella.
    if skipped_no_folder_num > 0:
        print(f"Attenzione: {skipped_no_folder_num} file sono stati saltati perché non è stato possibile estrarre il numero della sottocartella.")


#Controlla se la variabile 'train_pairs' esiste nello scope locale (locals()) e se non è vuota. Questo evita errori se 'image_mask_pairs' era vuoto e quindi 'train_pairs' non è stato definito.
if 'train_pairs' in locals() and train_pairs:
    #Chiama la funzione 'process_dataset_split' per elaborare le coppie del set di addestramento.
    process_dataset_split(train_pairs, 'train')
#stesso procedimento per il set di validazione
if 'val_pairs' in locals() and val_pairs:
    process_dataset_split(val_pairs, 'val')


# Crea il dizionario per il file YAML
# I percorsi nel YAML puntano alle cartelle
# Il file YAML è necessario per YOLOv5 e YOLOv8

#Inizia la definizione di un dizionario Python chiamato 'yaml_data'.
yaml_data = {
    ##prende i percorsi assoluti delle cartelle di addestramento e validazione
    'train': os.path.abspath(os.path.join(yolo_dataset_folder, 'images', 'train')),
    'val': os.path.abspath(os.path.join(yolo_dataset_folder, 'images', 'val')),
    'nc': 1, # Numero di classi
    'names': [class_name] # Lista dei nomi delle classi
}

#Costruisce il percorso completo per il file 'data.yaml' all'interno della cartella del dataset YOLO.
yaml_path = os.path.join(yolo_dataset_folder, 'data.yaml')
#Apre il file 'data.yaml' in modalità scrittura ('w').
with open(yaml_path, 'w') as f:
    #Scrive il contenuto del dizionario 'yaml_data' nel file 'f' (data.yaml) in formato YAML.
    # 'default_flow_style=False': produce un output YAML più leggibile (stile a blocchi invece che inline).
    # 'sort_keys=False': mantiene l'ordine delle chiavi come nel dizionario originale.
    yaml.dump(yaml_data, f, default_flow_style=False, sort_keys=False)

print(f"\nFile data.yaml creato in: {yaml_path}")
print("Contenuto del data.yaml:")
with open(yaml_path, 'r') as f:
  print(f.read())

print(f"\nPreprocessing completato! La cartella '{yolo_dataset_folder}' contiene le immagini e le etichette rinominate ed è pronta per l'addestramento YOLO.")

Cartella dataset originale: /content/dataset
Cartella dataset YOLO di output: /content/yolo_thyroid_dataset
Struttura cartelle YOLO creata.
Trovate 880 immagini originali.
Trovate 880 coppie immagine-maschera valide.
Suddivisione: 704 training, 176 validation.

Inizio processamento per lo split: train
  Processate 50/704 immagini per lo split train...
  Processate 100/704 immagini per lo split train...
  Processate 150/704 immagini per lo split train...
  Processate 200/704 immagini per lo split train...
  Processate 250/704 immagini per lo split train...
  Processate 300/704 immagini per lo split train...
  Processate 350/704 immagini per lo split train...
  Processate 400/704 immagini per lo split train...
  Processate 450/704 immagini per lo split train...
  Processate 500/704 immagini per lo split train...
  Processate 550/704 immagini per lo split train...
  Processate 600/704 immagini per lo split train...
Info: Trovati contorni in dataset/groundtruth02/unogt99.jpg, ma nessuno er

In [ ]:
#Utilizzando il gestore di pacchetti Python pip, installa la libreria ultralytics e, durante l'operazione,
# riduci al minimo i messaggi di output stampati sulla console (modalità silenziosa/quiet) utilizzando -q

!pip install ultralytics -q

In [ ]:
#Importa la classe YOLO dalla libreria 'ultralytics', che è la classe principale per utilizzare i modelli YOLOv8.
from ultralytics import YOLO
#Importa il modulo 'os', che fornisce funzioni per interagire con il sistema operativo, come la gestione dei percorsi dei file.
import os
#Importa la libreria PyTorch ('torch'). Viene qui utilizzata principalmente per verificare la disponibilità e le informazioni sulla GPU.
import torch

#Questo blocco di codice verifica la presenza di una GPU NVIDIA compatibile con CUDA.
print(f"GPU disponibile: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Nome GPU: {torch.cuda.get_device_name(0)}")

#Definisce la variabile 'data_yaml_path' con il percorso del file 'data.yaml'.
#Questo file è cruciale perché contiene le informazioni sul dataset
#(percorsi per training/validation, numero di classi, nomi delle classi) necessarie al modello YOLO per l'addestramento.
data_yaml_path = '/content/yolo_thyroid_dataset/data.yaml'

#Controlla se il file specificato in 'data_yaml_path' esiste nel filesystem.
if os.path.exists(data_yaml_path):
    print(f"File data.yaml trovato in: {data_yaml_path}")
    #Apre il file 'data.yaml' in modalità lettura ('r'). Il costrutto 'with' assicura la chiusura automatica del file.
    with open(data_yaml_path, 'r') as f:
         print("Contenuto di data.yaml:\n", f.read())
#Se il file 'data.yaml' non viene trovato al percorso specificato.
else:
    print(f"ERRORE: File data.yaml NON trovato in: {data_yaml_path}. Verifica il percorso!")


# Scegli il modello base per la segmentazione
# Opzioni comuni: 'yolov8n-seg.pt', 'yolov8s-seg.pt', 'yolov8m-seg.pt'
#Assegna alla variabile 'model_name' il nome del file del modello scelto, in questo caso 'yolov8s-seg.pt' (modello "small" per la segmentazione).
model_name = 'yolov8s-seg.pt'

#Crea un'istanza della classe YOLO, caricando i pesi del modello pre-addestrato specificato da 'model_name'. Questo oggetto 'model' verrà poi utilizzato per l'addestramento.
model = YOLO(model_name)

# --- Parametri di Addestramento ---
# Definisce il numero di 'epoche', ovvero il numero di volte in cui l'intero dataset di addestramento verrà processato dal modello.
epochs = 100
img_size = 640    # Dimensione delle immagini per l'addestramento (640x640 pixel è comune per YOLOv8)
#Definisce il 'batch_size', cioè il numero di immagini che vengono elaborate contemporaneamente dal modello prima di aggiornare i pesi. Questo valore deve essere adattato in base alla memoria GPU disponibile.
batch_size = 8
#Assegna un nome descrittivo ('run_name') a questa sessione di addestramento.
#Questo nome verrà utilizzato per creare una cartella dove verranno salvati i risultati (pesi, log, grafici).
run_name = 'thyroid_seg_finetune'

# Avvia l'addestramento!
# Assicurati che data_yaml_path sia corretto
if os.path.exists(data_yaml_path):
    print(f"\n--- Inizio Addestramento ---")
    # La variabile 'results' conterrà informazioni e metriche relative all'addestramento una volta completato.
    results = model.train(
        data=data_yaml_path,
        epochs=epochs,
        imgsz=img_size,
        batch=batch_size,
        name=run_name,  # Nome della cartella dei risultati
        project='YOLOv8_Thyroid_Runs' # Cartella radice per tutte le esecuzioni
    )

    print("--- Addestramento Completato ---")
else:
    print("Addestramento non avviato a causa del file data.yaml mancante.")

GPU disponibile: True
Nome GPU: Tesla T4
File data.yaml trovato in: /content/yolo_thyroid_dataset/data.yaml
Contenuto di data.yaml:
 train: /content/yolo_thyroid_dataset/images/train
val: /content/yolo_thyroid_dataset/images/val
nc: 1
names:
- thyroid



100%|██████████| 22.8M/22.8M [00:00<00:00, 121MB/s] 



--- Inizio Addestramento ---
Ultralytics 8.3.127 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/yolo_thyroid_dataset/data.yaml, degrees=0.0, deterministic=True, device=cuda:0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=thyroid_seg_finetune, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, pa

100%|██████████| 755k/755k [00:00<00:00, 24.9MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

...

    100/100      3.98G     0.2622     0.4239     0.1942     0.7984          7        640: 100%|██████████| 88/88 [00:18<00:00,  4.78it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:02<00:00,  5.30it/s]

                   all        176        146      0.999      0.993      0.995      0.966      0.999      0.993      0.995      0.922



100 epochs completed in 0.608 hours.
Optimizer stripped from YOLOv8_Thyroid_Runs/thyroid_seg_finetune/weights/last.pt, 23.9MB
Optimizer stripped from YOLOv8_Thyroid_Runs/thyroid_seg_finetune/weights/best.pt, 23.9MB

Validating YOLOv8_Thyroid_Runs/thyroid_seg_finetune/weights/best.pt...
Ultralytics 8.3.127 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8s-seg summary (fused): 85 layers, 11,779,987 parameters, 0 gradients, 42.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:04<00:00,  2.52it/s]


                   all        176        146          1      0.993      0.995      0.966          1      0.993      0.995       0.93
Speed: 0.2ms preprocess, 6.3ms inference, 0.0ms loss, 3.6ms postprocess per image
Results saved to YOLOv8_Thyroid_Runs/thyroid_seg_finetune
--- Addestramento Completato ---


In [19]:
!unrar x datasetTest.rar datasetTest


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from datasetTest.rar

Creating    datasetTest                                               OK
Creating    datasetTest/groundtruth01                                 OK
Extracting  datasetTest/groundtruth01/unogt100.jpeg                        0%  OK 
Extracting  datasetTest/groundtruth01/unogt140.jpeg                        0%  OK 
Extracting  datasetTest/groundtruth01/unogt141.jpeg                        0%  OK 
Extracting  datasetTest/groundtruth01/unogt142.jpeg                        0%  OK 
Extracting  datasetTest/groundtruth01/unogt143.jpeg                        0%  OK 
Extracting  datasetTest/groundtruth01/unogt144.jpeg                        0%  OK 
Extracting  datasetTest/groundtruth01/unogt145.jpeg                        0%  OK 
Extracting  datasetTest/groundtruth01/unogt146.jpeg                        0%  OK 
Extractin

In [ ]:
# il comando import importa moduli e pacchetti in Python.
'''
questo script serve a preprocessare i dati per la creazione di un dataset di test
per un modello YOLO (You Only Look Once) per la rilevazione di oggetti,
in questo caso per la tiroide.
Le immagini originali sono .jpg, le maschere sono .jpeg.
'''
#Importa la libreria OpenCV (cv2), utilizzata per l'elaborazione di immagini e video.
import cv2
#Importa la libreria NumPy e le assegna l'alias 'np'. NumPy è fondamentale per il calcolo numerico e la manipolazione di array.
import numpy as np
#Importa il modulo 'os', che fornisce funzioni per interagire con il sistema operativo, come la gestione di percorsi e directory.
import os
#Importa il modulo 'glob', utilizzato per trovare tutti i nomi di percorso che corrispondono a un pattern specificato.
import glob
#Dalla libreria Scikit-learn (sklearn), importa la funzione 'train_test_split', utile per suddividere i dati in set di addestramento e validazione.
# from sklearn.model_selection import train_test_split # Non più necessario per solo test set
#Importa la libreria 'yaml', utilizzata per leggere e scrivere file in formato YAML (YAML Ain't Markup Language), spesso usato per file di configurazione.
import yaml
#Importa il modulo 'shutil', che offre operazioni su file e collezioni di file di alto livello, come la copia e la rimozione di alberi di directory. Il commento originale indica già il suo uso per copiare i file.
import shutil
#Importa il modulo 're', che fornisce supporto per le espressioni regolari, utili per la ricerca e la manipolazione di stringhe basate su pattern.
import re

# --- CONFIGURAZIONE ---
base_dataset_folder = 'datasetTest' # Cartella che contiene 'originali01', 'groundtruth01' etc. del test set
yolo_dataset_folder = 'yolo_thyroid_dataset_test' # Cartella di output con file postprocess per il test set di YOLO
yaml_output_filename = 'data_test.yaml' # Nome del file YAML per il test set

# Definisci la regola per trovare la maschera dall'immagine originale
# Esempio: se l'immagine è 'datasetTest/originali01/uno100.jpg', la maschera è 'datasetTest/groundtruth01/unogt100.jpeg'
def get_mask_path(image_path):
    '''
    Firma della funzione: get_mask_path(image_path)
    - def: Parola chiave che definisce una funzione.
    - get_mask_path: Nome della funzione.
    - image_path: Parametro in input. È una stringa che rappresenta il percorso completo di un file immagine originale.
    Scopo: Questa funzione costruisce e restituisce il percorso presunto del file maschera corrispondente all'immagine fornita.
            Si basa su una convenzione specifica di denominazione e struttura delle cartelle:
            1. Estrae il numero identificativo dalla cartella dell'immagine originale (es. '01' da 'originali01').
            2. Usa questo numero per formare il nome della cartella delle maschere (es. 'groundtruth01').
            3. Modifica il nome del file immagine (es. sostituendo 'uno' con 'unogt') per ottenere il nome del file maschera.
            4. Assicura che l'estensione del file maschera sia .jpeg.
    Ritorna: Una stringa con il percorso completo al file maschera se la logica ha successo, altrimenti None se non è possibile determinare il percorso (es. se il numero della cartella non viene trovato).
    '''
    #Divide il percorso dell'immagine ('image_path') in una lista di componenti, utilizzando il separatore di directory specifico del sistema operativo (os.sep, es. '/' o '\').
    parts = image_path.split(os.sep)
    #Estrae l'ultimo elemento della lista 'parts', che corrisponde al nome del file con estensione (es. 'uno100.jpg').
    filename_orig_ext = parts[-1]
    # Estrai il numero dalla cartella originale (es. '01' da 'originali01')
    folder_name = parts[-2]
    #Utilizza un'espressione regolare (re.search) per trovare una sequenza di una o più cifre ('\d+') alla fine ('$') del nome della cartella ('folder_name').
    folder_num_match = re.search(r'\d+$', folder_name)
    #Controlla se l'espressione regolare ha trovato una corrispondenza (cioè, se è stato trovato un numero alla fine del nome della cartella).
    if folder_num_match:
        #Se una corrispondenza è stata trovata, 'folder_num_match.group()' restituisce la stringa corrispondente (es. '01').
        folder_num = folder_num_match.group()
        #Costruisce il nome della cartella delle maschere anteponendo 'groundtruth' al numero estratto (es. 'groundtruth01').
        mask_folder = f'groundtruth{folder_num}'

        # Estrai il nome base del file originale SENZA la sua estensione originale
        original_base_name, original_ext = os.path.splitext(filename_orig_ext) # es. filename_orig_ext='uno100.jpg' -> original_base_name='uno100'

        # Crea il nome base del file maschera (es. sostituendo 'uno' con 'unogt' se presente)
        mask_base_name = original_base_name.replace('uno', 'unogt')

        # *** Forza l'estensione del file maschera a .jpeg ***
        mask_filename_with_jpeg_ext = f"{mask_base_name}.jpeg"

        #Combina il percorso base del dataset, la cartella della maschera calcolata e il nome del file maschera calcolato per formare il percorso completo del file maschera, e lo restituisce.
        return os.path.join(os.path.dirname(os.path.dirname(image_path)), mask_folder, mask_filename_with_jpeg_ext)
    else:
        #Se l'espressione regolare non ha trovato un numero alla fine del nome della cartella.
        print(f"Attenzione: Impossibile determinare la cartella groundtruth per {image_path}")
        #Restituisce None per segnalare che il percorso della maschera non è stato trovato.
        return None

# Classe target (solo una: tiroide)
class_index = 0
class_name = 'thyroid'
# --- FINE CONFIGURAZIONE ---

print(f"Cartella dataset originale (test set): {os.path.abspath(base_dataset_folder)}")
print(f"Cartella dataset YOLO di output (test set): {os.path.abspath(yolo_dataset_folder)}")

# Pulisci la cartella di output se esiste già (opzionale, ma utile in Colab)
if os.path.exists(yolo_dataset_folder):
    print(f"Rimuovo la cartella YOLO (test) esistente: {yolo_dataset_folder}")
    shutil.rmtree(yolo_dataset_folder)

'''
Crea le cartelle necessarie per il test set di YOLO
le cartelle sono:
- images/test: per le immagini di test
- labels/test: per le etichette di test
'''
os.makedirs(os.path.join(yolo_dataset_folder, 'images', 'test'), exist_ok=True)
os.makedirs(os.path.join(yolo_dataset_folder, 'labels', 'test'), exist_ok=True)

print("Struttura cartelle YOLO per il test set creata.")

#Inizializza una lista vuota chiamata 'all_image_paths' che conterrà i percorsi di tutte le immagini originali trovate.
all_image_paths = []
# Trova tutte le immagini nelle cartelle 'originali*'
#Itera su tutte le cartelle all'interno di 'base_dataset_folder' il cui nome inizia con 'originali' (es. 'originali01', 'originali02').
for original_folder in glob.glob(os.path.join(base_dataset_folder, 'originali*')):
    # L'utente ha specificato che le immagini originali sono .jpg
    all_image_paths.extend(glob.glob(os.path.join(original_folder, '*.jpg')))
    # Manteniamo la flessibilità per altre estensioni se presenti, ma la priorità è .jpg per originali
    all_image_paths.extend(glob.glob(os.path.join(original_folder, '*.jpeg')))
    all_image_paths.extend(glob.glob(os.path.join(original_folder, '*.png')))

print(f"Trovate {len(all_image_paths)} immagini originali candidate nel test set.")

#Inizializza una lista vuota chiamata 'image_mask_pairs' per memorizzare le coppie di percorsi (immagine originale, maschera corrispondente).
image_mask_pairs = []
#Itera su ciascun percorso di immagine trovato e memorizzato in 'all_image_paths'.
for img_path in all_image_paths:
    #Chiama la funzione 'get_mask_path' per ottenere il percorso della maschera corrispondente all'immagine corrente.
    mask_path = get_mask_path(img_path)
    # Aggiungi solo se get_mask_path ha restituito un percorso valido e se esiste
    #Controlla due condizioni: 1) 'mask_path' non è None (cioè la funzione ha restituito un percorso) E 2) il file specificato da 'mask_path' esiste effettivamente.
    if mask_path and os.path.exists(mask_path):
        #Se entrambe le condizioni sono vere, aggiunge una tupla (coppia) contenente il percorso dell'immagine e il percorso della maschera alla lista 'image_mask_pairs'.
        image_mask_pairs.append((img_path, mask_path))
    elif mask_path:
        #Se 'mask_path' è stato restituito (non è None) ma il file non esiste (la condizione 'os.path.exists(mask_path)' era falsa).
        print(f"Attenzione: Maschera non trovata per {img_path} al percorso calcolato {mask_path} (Verifica estensione e nome file maschera!)")
    else: # mask_path è None
        print(f"Attenzione: get_mask_path non ha restituito un percorso per {img_path}")


print(f"Trovate {len(image_mask_pairs)} coppie immagine-maschera valide nel test set.")

#Controlla se la lista 'image_mask_pairs' è vuota (cioè, non sono state trovate coppie valide).
if not image_mask_pairs:
    print("Nessuna coppia immagine-maschera trovata. Controlla i percorsi, la funzione get_mask_path e le estensioni dei file.")
#else:
    # Non c'è suddivisione train/val, tutte le coppie sono per il test set

def process_dataset_split(pairs, split_name):
    '''
    Firma della funzione: process_dataset_split(pairs, split_name)
    - process_dataset_split: Nome della funzione.
    - pairs: Parametro in input. È una lista di tuple, dove ogni tupla contiene due stringhe: (percorso_immagine_originale, percorso_maschera_corrispondente).
    - split_name: Parametro in input. È una stringa che indica il nome del set di dati che si sta processando (es. "test").
    Scopo: Questa funzione elabora un insieme di coppie immagine-maschera per un determinato split.
            Per ogni coppia:
            1. Costruisce nuovi nomi per l'immagine e il file di etichetta, includendo un numero derivato dalla sottocartella originale per garantire unicità.
            2. Copia l'immagine originale nella sottocartella 'images/<split_name>' della directory YOLO con il nuovo nome.
            3. Legge l'immagine della maschera, la binarizza e trova i contorni della regione di interesse (es. la tiroide).
            4. Normalizza le coordinate dei punti del contorno rispetto alle dimensioni dell'immagine (valori tra 0 e 1).
            5. Salva i contorni normalizzati in un file di testo (formato etichetta YOLO) nella sottocartella 'labels/<split_name>' con il nuovo nome.
    Ritorna: Nulla (la funzione modifica il filesystem creando file e cartelle).
    '''
    print(f"\nInizio processamento per lo split: {split_name}")
    #Inizializza un contatore per le immagini processate con successo.
    processed_count = 0
    #Inizializza un contatore per i file saltati a causa dell'impossibilità di estrarre il numero della sottocartella.
    skipped_no_folder_num = 0

    #Itera su ogni tupla (percorso immagine, percorso maschera) nella lista 'pairs'.
    for img_path, mask_path in pairs:
        #Inizia un blocco try-except per gestire potenziali errori durante l'elaborazione di una singola coppia immagine-maschera.
        try:
            # --- Estrai informazioni per il nuovo nome ---
            #Estrae il nome base del file originale dell'immagine (es. "uno100.jpg") dal suo percorso completo.
            original_base_filename_with_ext = os.path.basename(img_path)
            #Separa il nome base del file nella sua parte nominale (es. "uno100") e la sua estensione (es. ".jpg").
            name_part, extension = os.path.splitext(original_base_filename_with_ext)

            # Divide il percorso completo dell'immagine originale nei suoi componenti.
            parts = img_path.split(os.sep)
            #Estrae il nome della cartella che contiene l'immagine (es. 'originali01').
            folder_name = parts[-2]
            #Utilizza un'espressione regolare per trovare una sequenza di cifre alla fine del nome della cartella.
            folder_num_match = re.search(r'\d+$', folder_name)

            #Se l'espressione regolare non trova un numero alla fine del nome della cartella.
            if not folder_num_match:
                print(f"Attenzione: Impossibile estrarre il numero dalla cartella '{folder_name}' per l'immagine {img_path}. Salto questo file.")
                #Incrementa il contatore dei file saltati per questo motivo.
                skipped_no_folder_num += 1
                #Interrompe l'iterazione corrente del ciclo e passa alla prossima coppia immagine-maschera.
                continue

            #Estrae la stringa numerica trovata (es. '01').
            subfolder_num_str = folder_num_match.group()

            # Costruisci i nuovi nomi dei file
            new_base_filename = f"{name_part}_{subfolder_num_str}{extension}" # Es: uno140_01.jpg (mantiene l'estensione originale dell'immagine)
            #Costruisce il nuovo nome del file etichetta, usando la stessa base del nuovo nome immagine ma con estensione ".txt" (es. "uno140_01.txt").
            new_label_filename = f"{name_part}_{subfolder_num_str}.txt"

            # --- Processa Immagine ---
            # Legge l'immagine originale dal percorso 'img_path' utilizzando OpenCV. 'img' sarà un array NumPy.
            img = cv2.imread(img_path)
            #Controlla se il caricamento dell'immagine è fallito (imread restituisce None in caso di errore).
            if img is None:
                print(f"Errore nel leggere l'immagine: {img_path}")
                #Salta all'iterazione successiva del ciclo.
                continue
            #Estrae altezza ('img_height') e larghezza ('img_width') dell'immagine dalle dimensioni dell'array NumPy ('img.shape').
            img_height, img_width = img.shape[:2]

            # Definisci il percorso di destinazione con il NUOVO nome
            yolo_img_path = os.path.join(yolo_dataset_folder, 'images', split_name, new_base_filename)
            #Copia il file immagine originale ('img_path') nella nuova posizione ('yolo_img_path'). 'copy2' tenta di preservare anche i metadati del file.
            shutil.copy2(img_path, yolo_img_path)

            # --- Processa Maschera e Crea Etichetta .txt ---
            #Legge l'immagine della maschera dal percorso 'mask_path' in scala di grigi (cv2.IMREAD_GRAYSCALE).
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            #Controlla se il caricamento della maschera è fallito.
            if mask is None:
                print(f"Errore nel leggere la maschera: {mask_path} (associata a {img_path})")
                #Se l'immagine era già stata copiata ma la sua maschera non è leggibile, rimuove l'immagine copiata per mantenere la coerenza.
                if os.path.exists(yolo_img_path): os.remove(yolo_img_path)
                #Salta all'iterazione successiva.
                continue

            #Applica una binarizzazione alla maschera. I pixel con valore > 60 diventano 255 (bianco), gli altri 0 (nero). Il primo valore restituito ('_') è la soglia usata, non ci serve quindi usiamo il simbolo '_' per ignorarlo.
            _, mask_binary = cv2.threshold(mask, 60, 255, cv2.THRESH_BINARY)

            #Trova i contorni nell'immagine della maschera binarizzata.
            contours, hierarchy = cv2.findContours(mask_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            #Costruisce il percorso completo per il file di etichetta .txt nella struttura YOLO, usando il nuovo nome file.
            yolo_label_path = os.path.join(yolo_dataset_folder, 'labels', split_name, new_label_filename)

            #Apre il file di etichetta in modalità scrittura ('w'). Il costrutto 'with' assicura che il file venga chiuso automaticamente. 'f' è l'oggetto file.
            with open(yolo_label_path, 'w') as f:
                #Inizializza un contatore per i contorni validi (quelli che soddisfano i criteri di dimensione/area) trovati in questa maschera.
                valid_contours_found = 0
                #Itera su ciascun contorno trovato nell'immagine della maschera.
                for contour in contours:
                    #Applica filtro su punti E area (opzionale ma consigliato)
                    min_contour_area = 5 # Ignora contorni con area < 5 pixel
                    #Controlla se il contorno è valido:
                    if contour.size >= 6 and cv2.contourArea(contour) > min_contour_area:
                        #'contour.flatten()' converte l'array del contorno in un array monodimensionale [x1,y1,x2,y2,...].
                        normalized_contour = contour.flatten().astype(float)
                        #Normalizza le coordinate x (elementi a indice pari: 0, 2, 4, ...) dividendo per la larghezza dell'immagine.
                        normalized_contour[0::2] /= img_width
                        #normalizza le coordinate y (elementi a indice dispari: 1, 3, 5, ...) dividendo per l'altezza dell'immagine.
                        normalized_contour[1::2] /= img_height
                        #Assicura che tutte le coordinate normalizzate siano strettamente comprese tra 0 e 1.
                        normalized_contour = np.clip(normalized_contour, 0.00001, 0.99999)
                        #Converte ogni coordinata normalizzata in una stringa formattata con 6 cifre decimali e poi unisce tutte queste stringhe con uno spazio.
                        contour_str = ' '.join(map(lambda x: f"{x:.6f}", normalized_contour))
                        #Scrive una riga nel file di etichetta.
                        f.write(f"{class_index} {contour_str}\n")
                        #Incrementa il contatore dei contorni validi.
                        valid_contours_found += 1
                #Se sono stati trovati dei contorni ma nessuno valido.
                if contours and valid_contours_found == 0:
                    print(f"Info: Trovati contorni in {mask_path}, ma nessuno era valido (>=3 punti e area > {min_contour_area}). File {new_label_filename} creato vuoto.")
                #Se non è stato trovato alcun contorno.
                elif not contours:
                    print(f"Info: Nessun contorno trovato in {mask_path}. File {new_label_filename} creato vuoto.")
                    pass

            #Incrementa il contatore delle immagini processate con successo in questo split.
            processed_count += 1
            #Controlla se il numero di immagini processate è un multiplo di 50.
            if processed_count % 50 == 0: # Stampa ogni 50 immagini
                print(f"   Processate {processed_count}/{len(pairs)} immagini per lo split {split_name}...")

        #Se si verifica un qualsiasi altro errore non gestito specificamente nel blocco 'try'. 'e' contiene l'oggetto eccezione.
        except Exception as e:
            print(f"Errore grave durante il processamento di {img_path} o {mask_path}: {e}")

    print(f"Fine processamento per lo split: {split_name}. Immagini processate correttamente: {processed_count}.")
    #Se alcuni file sono stati saltati a causa del problema con il numero della sottocartella.
    if skipped_no_folder_num > 0:
        print(f"Attenzione: {skipped_no_folder_num} file sono stati saltati perché non è stato possibile estrarre il numero della sottocartella.")


# Processa tutte le coppie trovate come 'test' set
if image_mask_pairs:
    process_dataset_split(image_mask_pairs, 'test')
else:
    print("Nessuna coppia immagine-maschera da processare per il test set.")


# Crea il dizionario per il file YAML del test set
yaml_data = {
    'test': os.path.abspath(os.path.join(yolo_dataset_folder, 'images', 'test')),
    'train': os.path.abspath(os.path.join(yolo_dataset_folder, 'images', 'test')),
    'val': os.path.abspath(os.path.join(yolo_dataset_folder, 'images', 'test')),
    'nc': 1, # Numero di classi
    'names': [class_name] # Lista dei nomi delle classi
}

#Costruisce il percorso completo per il file yaml_output_filename all'interno della cartella del dataset YOLO di test.
yaml_path = os.path.join(yolo_dataset_folder, yaml_output_filename)
#Apre il file YAML in modalità scrittura ('w').
with open(yaml_path, 'w') as f:
    #Scrive il contenuto del dizionario 'yaml_data' nel file 'f' in formato YAML.
    yaml.dump(yaml_data, f, default_flow_style=False, sort_keys=False)

print(f"\nFile {yaml_output_filename} creato in: {yaml_path}")
print(f"Contenuto del {yaml_output_filename}:")
with open(yaml_path, 'r') as f:
    print(f.read())

print(f"\nPreprocessing per il test set completato! La cartella '{yolo_dataset_folder}' contiene le immagini e le etichette per il test ed è pronta per la valutazione YOLO.")

Cartella dataset originale (test set): /content/datasetTest
Cartella dataset YOLO di output (test set): /content/yolo_thyroid_dataset_test
Rimuovo la cartella YOLO (test) esistente: yolo_thyroid_dataset_test
Struttura cartelle YOLO per il test set creata.
Trovate 330 immagini originali candidate nel test set.
Trovate 330 coppie immagine-maschera valide nel test set.

Inizio processamento per lo split: test
Info: Nessun contorno trovato in datasetTest/groundtruth15/unogt230.jpeg. File uno230_15.txt creato vuoto.
Info: Nessun contorno trovato in datasetTest/groundtruth15/unogt228.jpeg. File uno228_15.txt creato vuoto.
Info: Nessun contorno trovato in datasetTest/groundtruth15/unogt226.jpeg. File uno226_15.txt creato vuoto.
Info: Nessun contorno trovato in datasetTest/groundtruth15/unogt229.jpeg. File uno229_15.txt creato vuoto.
Info: Nessun contorno trovato in datasetTest/groundtruth15/unogt223.jpeg. File uno223_15.txt creato vuoto.
Info: Nessun contorno trovato in datasetTest/groundtrut

In [31]:
from ultralytics import YOLO
import torch
import os
import pandas as pd # Importa pandas
import numpy as np  # Importa numpy per np.number
import traceback    # Per un debug più dettagliato

# --- Configurazione ---
PATH_AL_FILE_YAML_DEL_TEST_SET = '/content/yolo_thyroid_dataset_test/data_test.yaml' # Assicurati che questo YAML usi PERCORSI ASSOLUTI
PATH_AI_PESI_FINETUNED = 'best.pt'
OUTPUT_EXCEL_FILE = 'risultati_valutazione_yolo_finetuned.xlsx'

IMG_SIZE = 640
BATCH_SIZE = 8
CONF_THRESHOLD = 0.25
IOU_THRESHOLD = 0.5
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Lista per conservare i dati per Excel
excel_data_rows = []

# Verifica l'esistenza del file YAML
if not os.path.exists(PATH_AL_FILE_YAML_DEL_TEST_SET):
    print(f"ERRORE: File YAML del test set non trovato in '{PATH_AL_FILE_YAML_DEL_TEST_SET}'")
    exit()
else:
    print(f"File YAML del test set trovato: '{PATH_AL_FILE_YAML_DEL_TEST_SET}'. Assicurati che usi percorsi assoluti per i dataset.")

# --- Valutare il tuo modello fine-tuned (best.pt) sul tuo test set ---
print(f"\n\n--- Inizio Valutazione del Modello Fine-tuned ({PATH_AI_PESI_FINETUNED}) sul Test Set ---")
metrics_finetuned_data_for_excel = {"Modello": PATH_AI_PESI_FINETUNED, "Numero Immagini": 'N/A'} # Inizializza

if not os.path.exists(PATH_AI_PESI_FINETUNED):
    print(f"ERRORE: File dei pesi fine-tuned non trovato in '{PATH_AI_PESI_FINETUNED}'")
    metrics_finetuned_data_for_excel["mAP@0.5:0.95 (seg)"] = "Pesi non trovati"
    metrics_finetuned_data_for_excel["mAP@0.5 (seg)"] = "Pesi non trovati"
    metrics_finetuned_data_for_excel["Precision (seg)"] = "Pesi non trovati"
    metrics_finetuned_data_for_excel["Recall (seg)"] = "Pesi non trovati"
    excel_data_rows.append(metrics_finetuned_data_for_excel)
else:
    try:
        model_finetuned = YOLO(PATH_AI_PESI_FINETUNED)
        model_finetuned.to(DEVICE)

        print(f"Avvio di model.val() per il modello fine-tuned...")
        metrics_finetuned_obj = model_finetuned.val(
            data=PATH_AL_FILE_YAML_DEL_TEST_SET, # Assicurati che questo YAML usi percorsi assoluti
            imgsz=IMG_SIZE,
            batch=BATCH_SIZE,
            conf=CONF_THRESHOLD,
            iou=IOU_THRESHOLD,
            split='test',
            device=DEVICE,
            project='YOLOv8_Finetuned_Eval_Runs', # I risultati di questa esecuzione verranno salvati qui
            name='eval_finetuned_thyroid_test'  # Puoi cambiare questo nome per ogni esecuzione se vuoi tenere traccia
        )

        print("\n--- Risultati Valutazione Modello Fine-tuned (da results_dict) ---")

        if hasattr(metrics_finetuned_obj, 'results_dict'):
            results_d_finetuned = metrics_finetuned_obj.results_dict

            # Estrai il numero di immagini
            num_images_finetuned = metrics_finetuned_obj.dataset.len if hasattr(metrics_finetuned_obj, 'dataset') and hasattr(metrics_finetuned_obj.dataset, 'len') else 'N/A'
            if num_images_finetuned == 'N/A':
                 # Fallback se .dataset.len non è disponibile (es. da output precedente 330)
                 # Potresti inserire un valore noto o lasciare 'N/A'
                 print("Numero immagini non trovato tramite metrics_finetuned_obj.dataset.len. Controlla output della tabella YOLO.")


            metrics_finetuned_data_for_excel["Numero Immagini"] = num_images_finetuned
            metrics_finetuned_data_for_excel["mAP@0.5:0.95 (seg)"] = results_d_finetuned.get('metrics/mAP50-95(M)')
            metrics_finetuned_data_for_excel["mAP@0.5 (seg)"] = results_d_finetuned.get('metrics/mAP50(M)')
            metrics_finetuned_data_for_excel["Precision (seg)"] = results_d_finetuned.get('metrics/precision(M)')
            metrics_finetuned_data_for_excel["Recall (seg)"] = results_d_finetuned.get('metrics/recall(M)')

            # Stampa le metriche che andranno in Excel
            for key, value in metrics_finetuned_data_for_excel.items():
                if isinstance(value, (float, np.number)):
                     print(f"  {key}: {value:.4f}")
                else:
                     print(f"  {key}: {value}")

            excel_data_rows.append(metrics_finetuned_data_for_excel)
        else:
            print("  ERRORE: metrics_finetuned_obj.results_dict non trovato. Impossibile estrarre le metriche.")
            metrics_finetuned_data_for_excel["mAP@0.5:0.95 (seg)"] = "Errore (no results_dict)"
            metrics_finetuned_data_for_excel["mAP@0.5 (seg)"] = "Errore (no results_dict)"
            metrics_finetuned_data_for_excel["Precision (seg)"] = "Errore (no results_dict)"
            metrics_finetuned_data_for_excel["Recall (seg)"] = "Errore (no results_dict)"
            excel_data_rows.append(metrics_finetuned_data_for_excel)


    except Exception as e:
        print(f"Errore GRAVE durante la valutazione del modello fine-tuned: {e}")
        traceback.print_exc()
        metrics_finetuned_data_for_excel["mAP@0.5:0.95 (seg)"] = "Errore (eccezione)"
        metrics_finetuned_data_for_excel["mAP@0.5 (seg)"] = "Errore (eccezione)"
        metrics_finetuned_data_for_excel["Precision (seg)"] = "Errore (eccezione)"
        metrics_finetuned_data_for_excel["Recall (seg)"] = "Errore (eccezione)"
        excel_data_rows.append(metrics_finetuned_data_for_excel)

# --- Salvataggio in Excel ---
if excel_data_rows:
    df = pd.DataFrame(excel_data_rows)
    # Definisci l'ordine desiderato delle colonne
    col_order = ["Modello", "Numero Immagini", "mAP@0.5:0.95 (seg)", "mAP@0.5 (seg)", "Precision (seg)", "Recall (seg)"]
    # Filtra le colonne per mantenere solo quelle presenti nel DataFrame e nell'ordine specificato
    df_cols_existing = [col for col in col_order if col in df.columns]
    df = df[df_cols_existing]

    try:
        df.to_excel(OUTPUT_EXCEL_FILE, index=False, engine='openpyxl')
        print(f"\nMetriche del modello fine-tuned salvate con successo nel file Excel: '{OUTPUT_EXCEL_FILE}'")
    except Exception as e:
        print(f"\nErrore durante il salvataggio del file Excel: {e}")
        print("Assicurati di avere installato 'openpyxl': pip install openpyxl")
else:
    print("\nNessun dato di metrica del modello fine-tuned da salvare nel file Excel.")

print("\n--- Valutazione Completata ---")

File YAML del test set trovato: '/content/yolo_thyroid_dataset_test/data_test.yaml'. Assicurati che usi percorsi assoluti per i dataset.


--- Inizio Valutazione del Modello Fine-tuned (best.pt) sul Test Set ---
Avvio di model.val() per il modello fine-tuned...
YOLOv8s-seg summary (fused): 85 layers, 11,779,987 parameters, 0 gradients, 42.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 597.0±226.4 MB/s, size: 19.9 KB)


val: Scanning /content/yolo_thyroid_dataset_test/labels/test.cache... 330 images, 63 backgrounds, 0 corrupt: 100%|██████████| 330/330 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 42/42 [00:05<00:00,  7.38it/s]


                   all        330        268      0.975       0.44      0.712      0.478      0.975       0.44      0.712      0.437
Speed: 0.7ms preprocess, 11.3ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to YOLOv8_Finetuned_Eval_Runs/eval_finetuned_thyroid_test8

--- Risultati Valutazione Modello Fine-tuned (da results_dict) ---
Numero immagini non trovato tramite metrics_finetuned_obj.dataset.len. Controlla output della tabella YOLO.
  Modello: best.pt
  Numero Immagini: N/A
  mAP@0.5:0.95 (seg): 0.4375
  mAP@0.5 (seg): 0.7117
  Precision (seg): 0.9752
  Recall (seg): 0.4403

Metriche del modello fine-tuned salvate con successo nel file Excel: 'risultati_valutazione_yolo_finetuned.xlsx'

--- Valutazione Completata ---


In [32]:
! ls /content/yolo_thyroid_dataset_test/images/test/ | wc -l

330
